In [34]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [35]:
def get_UCIHAR(paths):
    colnames = ['subject', 'x_acc', 'y_acc', 'z_acc', 'x_gyro', 'y_gyro', 'z_gyro', 'x_total_acc', 'y_total_acc', 'z_total_acc', 'activity']
    cols = {col: None for col in colnames}
    for i, path in enumerate(paths):
        col_data = []
        with open(path) as file:
            for row in file:
                if (i == 0 or i == 10):
                    col_data.append(int(row))
                else:
                    data = re.findall(r'[\\+|-]?[1-9]\.[0-9]+e[+|-][0-9]+', row)
                    data_vec = np.array([float(element) for element in data])
                    col_data.append(data_vec)
        cols[colnames[i]] = col_data
    data = pd.DataFrame.from_dict(cols)
    return data
                

In [36]:
train_paths = ['../Data/UCI HAR Dataset/train/subject_train.txt',
                '../Data/UCI HAR Dataset/train/body_acc_x_train.txt',
                '../Data/UCI HAR Dataset/train/body_acc_y_train.txt', 
                '../Data/UCI HAR Dataset/train/body_acc_z_train.txt',
                '../Data/UCI HAR Dataset/train/body_gyro_x_train.txt',
                '../Data/UCI HAR Dataset/train/body_gyro_y_train.txt',
                '../Data/UCI HAR Dataset/train/body_gyro_z_train.txt',
                '../Data/UCI HAR Dataset/train/total_acc_x_train.txt',
                '../Data/UCI HAR Dataset/train/total_acc_y_train.txt', 
                '../Data/UCI HAR Dataset/train/total_acc_z_train.txt',
                '../Data/UCI HAR Dataset/train/y_train.txt'
              ]

test_paths = ['../Data/UCI HAR Dataset/test/subject_test.txt',
                '../Data/UCI HAR Dataset/test/body_acc_x_test.txt',
                '../Data/UCI HAR Dataset/test/body_acc_y_test.txt', 
                '../Data/UCI HAR Dataset/test/body_acc_z_test.txt',
                '../Data/UCI HAR Dataset/test/body_gyro_x_test.txt',
                '../Data/UCI HAR Dataset/test/body_gyro_y_test.txt',
                '../Data/UCI HAR Dataset/test/body_gyro_z_test.txt',
                '../Data/UCI HAR Dataset/test/total_acc_x_test.txt',
                '../Data/UCI HAR Dataset/test/total_acc_y_test.txt', 
                '../Data/UCI HAR Dataset/test/total_acc_z_test.txt',
                '../Data/UCI HAR Dataset/test/y_test.txt'
              ]
train_data = get_UCIHAR(train_paths)
test_data = get_UCIHAR(test_paths)

In [37]:
cols = ['x_acc', 'y_acc', 'z_acc', 'x_gyro', 'y_gyro', 'z_gyro', 'x_total_acc', 'y_total_acc', 'z_total_acc']
def build_features(data):
    features = np.empty(shape = (len(data), 9, 128))
    
    for i, row in data.iterrows():
        features[i, :, :] = transfer_column(row)
    return features    
def transfer_column(row):
    transfer_row = np.vstack([row[col] for col in cols])
    return transfer_row

In [38]:
train_features = build_features(train_data)
train_labels = train_data['activity'].to_numpy()
train_features_fft = np.abs(np.fft.fft2(train_features))

test_features = build_features(test_data)
test_labels = test_data['activity'].to_numpy()
test_features_fft = np.abs(np.fft.fft2(test_features))

In [39]:
def reshape_data(features, n):
    features = np.reshape(features, (n, 128, 9))
    return features

In [40]:
from tensorflow.keras.utils import to_categorical

In [41]:
def one_hot(y):
    y = y-1
    y = to_categorical(y)
    return y

In [42]:
train_features = reshape_data(train_features, 7352)
test_features = reshape_data(test_features, 2947)
train_labels = one_hot(train_labels)
test_labels = one_hot(test_labels)

In [50]:
np.shape(train_labels)

(7352, 6)

In [55]:
def model(train_features, train_labels, test_features, test_labels):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = train_features.shape[1], train_features.shape[2], train_labels.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size, verbose=verbose)
    _, accuracy = model.evaluate(test_features, test_labels, batch_size=batch_size, verbose=0)
    return accuracy

In [56]:
def summarise(accuracy_scores):
    print(accuracy_scores)
    m, s = mean(accuracy_scores), std(accuracy_scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [57]:
def run_experiment(train_features, train_labels, test_features, test_labels, repeats=10):
    scores = list()
    for r in range(repeats):
        accuracy_score = model(train_features, train_labels, test_features, test_labels)
        accuracy_score = accuracy_score * 100.0
        print('>#%d: %.3f' % (r+1, accuracy_score))
        scores.append(accuracy_score)
    # summarize results
    summarise(scores)

In [58]:
run_experiment(train_features, train_labels, test_features, test_labels)

>#1: 81.371
>#2: 82.457
>#3: 80.692
>#4: 83.203
>#5: 80.930
>#6: 83.610
>#7: 85.952
>#8: 83.950
>#9: 82.423
>#10: 82.185
[81.37088418006897, 82.45673775672913, 80.69223165512085, 83.2032561302185, 80.92975616455078, 83.61045122146606, 85.95181703567505, 83.94978046417236, 82.4228048324585, 82.18527436256409]
Accuracy: 82.677% (+/-1.502)
